# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f1e80c62250>
├── 'a' --> tensor([[ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088]])
└── 'x' --> <FastTreeValue 0x7f1e80c62df0>
    └── 'c' --> tensor([[ 0.5303,  0.3973,  0.3508, -0.0522],
                        [-0.5450, -0.0214, -1.5910, -0.3621],
                        [ 0.9580, -1.6776, -0.0139, -1.2663]])

In [4]:
t.a

tensor([[ 0.9719, -0.7586, -1.1240],
        [-2.8899,  0.2374, -0.2088]])

In [5]:
%timeit t.a

71.9 ns ± 1.58 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f1e80c62250>
├── 'a' --> tensor([[-0.0698, -1.5717,  1.2439],
│                   [-0.7451, -1.7621, -0.6387]])
└── 'x' --> <FastTreeValue 0x7f1e80c62df0>
    └── 'c' --> tensor([[ 0.5303,  0.3973,  0.3508, -0.0522],
                        [-0.5450, -0.0214, -1.5910, -0.3621],
                        [ 0.9580, -1.6776, -0.0139, -1.2663]])

In [7]:
%timeit t.a = new_value

71.6 ns ± 2.5 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.9719, -0.7586, -1.1240],
               [-2.8899,  0.2374, -0.2088]]),
    x: Batch(
           c: tensor([[ 0.5303,  0.3973,  0.3508, -0.0522],
                      [-0.5450, -0.0214, -1.5910, -0.3621],
                      [ 0.9580, -1.6776, -0.0139, -1.2663]]),
       ),
)

In [10]:
b.a

tensor([[ 0.9719, -0.7586, -1.1240],
        [-2.8899,  0.2374, -0.2088]])

In [11]:
%timeit b.a

71.5 ns ± 0.793 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 0.6148,  0.4555, -0.2663],
               [-1.3588, -1.8068, -2.1225]]),
    x: Batch(
           c: tensor([[ 0.5303,  0.3973,  0.3508, -0.0522],
                      [-0.5450, -0.0214, -1.5910, -0.3621],
                      [ 0.9580, -1.6776, -0.0139, -1.2663]]),
       ),
)

In [13]:
%timeit b.a = new_value

571 ns ± 13 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

909 ns ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.5 µs ± 215 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

274 µs ± 5.5 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

264 µs ± 7.81 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f1e80ccfd00>
├── 'a' --> tensor([[[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]],
│           
│                   [[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]],
│           
│                   [[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]],
│           
│                   [[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]],
│           
│                   [[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]],
│           
│                   [[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]],
│           
│                   [[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]],
│           
│                   [[ 0.9719, -0.7586, -1.1240],
│                    [-2.8899,  0.2374, -0.2088]]])
└── 'x' --> <FastTreeValue 0x7f1dc9551250>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

42.4 µs ± 570 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f1e80c622b0>
├── 'a' --> tensor([[ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088],
│                   [ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088],
│                   [ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088],
│                   [ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088],
│                   [ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088],
│                   [ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088],
│                   [ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088],
│                   [ 0.9719, -0.7586, -1.1240],
│                   [-2.8899,  0.2374, -0.2088]])
└── 'x' --> <FastTreeValue 0x7f1e80c62970>
    └── 'c' --> tensor([[ 0.5303,  0.3973,  0.3508, -0.0522],
                        [-0.5450, -0.0214, -1.5910, -0.3621],
                 

In [23]:
%timeit t_cat(trees)

39.1 µs ± 817 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

76 µs ± 743 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 0.5303,  0.3973,  0.3508, -0.0522],
                       [-0.5450, -0.0214, -1.5910, -0.3621],
                       [ 0.9580, -1.6776, -0.0139, -1.2663]],
              
                      [[ 0.5303,  0.3973,  0.3508, -0.0522],
                       [-0.5450, -0.0214, -1.5910, -0.3621],
                       [ 0.9580, -1.6776, -0.0139, -1.2663]],
              
                      [[ 0.5303,  0.3973,  0.3508, -0.0522],
                       [-0.5450, -0.0214, -1.5910, -0.3621],
                       [ 0.9580, -1.6776, -0.0139, -1.2663]],
              
                      [[ 0.5303,  0.3973,  0.3508, -0.0522],
                       [-0.5450, -0.0214, -1.5910, -0.3621],
                       [ 0.9580, -1.6776, -0.0139, -1.2663]],
              
                      [[ 0.5303,  0.3973,  0.3508, -0.0522],
                       [-0.5450, -0.0214, -1.5910, -0.3621],
                       [ 0.9580, -1.6776, -0.0139, -1.2663]],

In [26]:
%timeit Batch.stack(batches)

93.5 µs ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 0.5303,  0.3973,  0.3508, -0.0522],
                      [-0.5450, -0.0214, -1.5910, -0.3621],
                      [ 0.9580, -1.6776, -0.0139, -1.2663],
                      [ 0.5303,  0.3973,  0.3508, -0.0522],
                      [-0.5450, -0.0214, -1.5910, -0.3621],
                      [ 0.9580, -1.6776, -0.0139, -1.2663],
                      [ 0.5303,  0.3973,  0.3508, -0.0522],
                      [-0.5450, -0.0214, -1.5910, -0.3621],
                      [ 0.9580, -1.6776, -0.0139, -1.2663],
                      [ 0.5303,  0.3973,  0.3508, -0.0522],
                      [-0.5450, -0.0214, -1.5910, -0.3621],
                      [ 0.9580, -1.6776, -0.0139, -1.2663],
                      [ 0.5303,  0.3973,  0.3508, -0.0522],
                      [-0.5450, -0.0214, -1.5910, -0.3621],
                      [ 0.9580, -1.6776, -0.0139, -1.2663],
                      [ 0.5303,  0.3973,  0.3508, -0.0522],
                   

In [28]:
%timeit Batch.cat(batches)

173 µs ± 3.14 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

550 µs ± 12.3 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
